## Requirements

In [4]:
import numpy as np

## Problem setting

To illustrate that algorithms have a potentially large inflcuence on run times, you can consider the matrix power operation, i.e., for a square matrix $A$, compute
$$
    A^n = A \cdot A \cdot \cdots A
$$

## Implementation

### Naive implementation

The following implementation is the baseline, it simply multiplies the given matrix $n$ times.

In [8]:
def naive_exponentiation(A, n):
    result = np.eye(A.shape[0])
    for _ in range(n):
        result = result@A
    return result

It is easy to verify that the implementation produces the correct result.

In [9]:
A = np.array([
    [1, 2],
    [3, 5],
])

In [10]:
naive_exponentiation(A, 2)

array([[ 7., 12.],
       [18., 31.]])

### Binary exponentiation

The following example can help you realize that $n$ multiplication are not required.
$$
    A^13 = A^8 \cdot A^4 \cdot A
$$
where
$$
    \begin{array}{lcr}
        A_2 & = & A \cdot A \\
        A_4 & = & A_2 \cdot A_2 \\
        A_8 & = & A_4 \cdot A_4 \\
        A^13 = A_8 \cdot A_4 \cdot A
    \end{array}
$$
So rather tha 13 multiplications in the naive implementation, you only need 5 multiplications using this approach.

The crux is to convert the exponent to binary, e.g., $13 = 2^4 + 2^2 + 1 = 1101_2$

In [11]:
def binary_exponentiation(A, n):
    powers = []
    factors = []
    factor = None
    idx = 0
    while n > 0:
        factors.append(factors[-1]@factors[-1] if factors else A)
        if n & 1:
            powers.append(idx)
        n >>= 1
        idx += 1
    result = None
    for idx in powers:
        factor = factors[idx]
        result = factor if result is None else result@factor
    return result

The result is identical to that of the naive implementation.

In [12]:
binary_exponentiation(A, 3)

array([[ 43,  74],
       [111, 191]])

In [13]:
naive_exponentiation(A, 3)

array([[ 43.,  74.],
       [111., 191.]])

## Performance

To test the performance of both implementations, you would use a large(r) matrix and a larger exponent.

In [14]:
A = np.random.normal(size=(2_000, 2_000))

In [15]:
%timeit binary_exponentiation(A, 31)

807 ms ± 74.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit naive_exponentiation(A, 31)

2.76 s ± 44.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit binary_exponentiation(A, 32)

494 ms ± 26.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit naive_exponentiation(A, 32)

2.84 s ± 48.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It is clear that the binary exponentiation is significantly faster than the naive implementation.

However, the numpy implementation is about 10 % faster.

In [19]:
%timeit np.linalg.matrix_power(A, 31)

759 ms ± 34.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit np.linalg.matrix_power(A, 32)

450 ms ± 8.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
